In [50]:
!pip3 install deap
!pip3 install pyswarms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import numpy as np
import random
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
from pyswarms.single import GlobalBestPSO
from secrets import randbelow

In [52]:
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [53]:
data.drop(['ZIP Code','ID'],axis=1,inplace=True)

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = data.drop('Personal Loan',axis=1)
y = data['Personal Loan']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [55]:
print(len(data))
print("No. of columns : ", 11)

5000
No. of columns :  11


# PyTorch Neural Networks + Genetic Algorithms

In [56]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

'''
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

'''


def create_population(pop_size, input_size, hidden_size, output_size):
    population = []
    for i in range(pop_size):
        net = NeuralNet(input_size, hidden_size, output_size)
        for param in net.parameters():
            param.data = torch.from_numpy(np.random.uniform(-1, 1, size=param.data.size())).float()
        population.append(net)
    return population


def fitness_function(net, data, target):
    inputs = torch.from_numpy(data).float()
    targets = torch.from_numpy(target).float()
    outputs = net(inputs)
    loss = torch.nn.BCELoss()(outputs, targets)
    acc = ((outputs > 0.5) == targets).sum().item() / len(outputs)
    return acc, loss


pop_size = 50
num_generations = 50
mutation_rate = 0.1
input_size = 20
hidden_size = 6
output_size = 1
data = np.random.rand(981, 20)
target = np.random.randint(2, size=(981, 1))


population = create_population(pop_size, input_size, hidden_size, output_size)

for generation in range(num_generations):
    fitness_scores = []
    for i in range(pop_size):
        net = population[i]
        acc, loss = fitness_function(net, data, target)
        fitness_scores.append((i, acc))

    fitness_scores.sort(key=lambda x: x[1], reverse=True)

    best_individual = population[fitness_scores[0][0]]
    best_acc, best_loss = fitness_function(best_individual, data, target)
    print("Generation:", generation, "Best accuracy:", best_acc)

    parents = [population[i] for i in range(pop_size//2)]

    offspring = []
    for i in range(pop_size//2):
        parent1 = parents[np.random.randint(len(parents))]
        parent2 = parents[np.random.randint(len(parents))]
        child = NeuralNet(input_size, hidden_size, output_size)
        for c, p1, p2 in zip(child.parameters(), parent1.parameters(), parent2.parameters()):
            c.data = torch.where(torch.rand_like(c.data) > 0.5, p1.data, p2.data)
        offspring.append(child)

    for i in range(len(offspring)):
        child = offspring[i]
        for param in child.parameters():
            if np.random.rand() < mutation_rate:
                param.data += torch.from_numpy(np.random.normal(scale=0.1, size=param.data.size())).float()

    for i in range(pop_size//2):
        population[-(i+1)] = offspring[i]


    fitness_scores = []
    for i in range(pop_size):
      net = population[i]
      acc, loss = fitness_function(net, data, target)
      fitness_scores.append((i, acc))


    fitness_scores.sort(key=lambda x: x[1], reverse=True)


best_individual = population[fitness_scores[0][0]]
best_acc, best_loss = fitness_function(best_individual, data, target)
print("Best accuracy:", best_acc)



Generation: 0 Best accuracy: 0.5219164118246687
Generation: 1 Best accuracy: 0.5219164118246687
Generation: 2 Best accuracy: 0.5300713557594292
Generation: 3 Best accuracy: 0.528032619775739
Generation: 4 Best accuracy: 0.5341488277268094
Generation: 5 Best accuracy: 0.5219164118246687
Generation: 6 Best accuracy: 0.5361875637104995
Generation: 7 Best accuracy: 0.5219164118246687
Generation: 8 Best accuracy: 0.5219164118246687
Generation: 9 Best accuracy: 0.5219164118246687
Generation: 10 Best accuracy: 0.5219164118246687
Generation: 11 Best accuracy: 0.5219164118246687
Generation: 12 Best accuracy: 0.5219164118246687
Generation: 13 Best accuracy: 0.5219164118246687
Generation: 14 Best accuracy: 0.5310907237512742
Generation: 15 Best accuracy: 0.5219164118246687
Generation: 16 Best accuracy: 0.5219164118246687
Generation: 17 Best accuracy: 0.5219164118246687
Generation: 18 Best accuracy: 0.5361875637104995
Generation: 19 Best accuracy: 0.5249745158002038
Generation: 20 Best accuracy: 0

# Particle Swarm Optimization (PySwarms Module)

In [57]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out


input_size = 20
hidden_size = 10
output_size = 1

net = NeuralNet(input_size, hidden_size, output_size)

# params = list(net.parameters())
# print(params)
# print(params[0].size())

# Get the list of weights
weights = []
for name, param in net.named_parameters():
    if 'weight' in name:
        weights.append(param.data.tolist())



print(len(weights))

print("\n\n")

for i in weights :
  print(len(i))


print("\n\n")

for i in weights :
  for j in i :
    print(len(j))

2



10
1



20
20
20
20
20
20
20
20
20
20
10


In [58]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

input_size = 20
hidden_size = 10
output_size = 1

net = NeuralNet(input_size, hidden_size, output_size)

def fitness_function(weights):
    with torch.no_grad():
        # weights = weights.reshape(hidden_size * hidden_size, input_size+1)
        weights = torch.Tensor(weights)

        net.fc1.weight.data = weights[:, :input_size]
        net.fc1.bias.data = weights[:, input_size]

        net.fc2.weight.data = torch.Tensor(np.random.rand(1, hidden_size))
        net.fc2.bias.data = torch.Tensor(np.random.rand(1))

        x_test = torch.Tensor(np.random.rand(1, input_size))
        y_pred = net(x_test)
        loss = torch.nn.MSELoss()(y_pred, torch.Tensor([[1.0]]))
        
    return loss.item()

bounds = (np.zeros(hidden_size*(input_size+1)), np.ones(hidden_size*(input_size+1)))

options = {"c1":0.5, "c2":0.3, "w":0.9}
optimizer = GlobalBestPSO(n_particles=10, dimensions=hidden_size*(input_size+1), options=options)
best_weights, best_fitness = optimizer.optimize(fitness_function, iters=100)

print("Best weights:", best_weights)
print("Best fitness:", best_fitness)

2023-04-05 03:59:20,810 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0
2023-04-05 03:59:21,318 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [5.36200693e-01 5.31850479e-01 5.71121321e-01 4.58514063e-01
 2.89105332e-01 2.69881428e-01 4.14517147e-01 4.68812648e-01
 7.17999175e-01 8.29097401e-01 9.56662281e-01 5.01001212e-01
 1.52131062e-01 4.86674010e-01 5.50475221e-01 7.84799654e-01
 7.02079961e-01 8.05328168e-01 1.88704879e-01 9.72985530e-01
 5.22437727e-01 3.66472972e-01 4.10575922e-01 1.36487687e-02
 7.65556391e-01 2.00115949e-01 3.56339637e-01 3.13531681e-01
 4.15291273e-01 6.71767330e-01 7.70523337e-01 3.72879320e-01
 6.85258651e-01 5.97205926e-01 8.31775710e-01 4.23683551e-01
 7.22733349e-01 8.35692044e-02 2.30596330e-01 2.41426880e-01
 5.43136613e-01 4.44868536e-01 5.28540519e-01 2.77622762e-01
 1.50708046e-01 7.28094073

Best weights: 0.0
Best fitness: [5.36200693e-01 5.31850479e-01 5.71121321e-01 4.58514063e-01
 2.89105332e-01 2.69881428e-01 4.14517147e-01 4.68812648e-01
 7.17999175e-01 8.29097401e-01 9.56662281e-01 5.01001212e-01
 1.52131062e-01 4.86674010e-01 5.50475221e-01 7.84799654e-01
 7.02079961e-01 8.05328168e-01 1.88704879e-01 9.72985530e-01
 5.22437727e-01 3.66472972e-01 4.10575922e-01 1.36487687e-02
 7.65556391e-01 2.00115949e-01 3.56339637e-01 3.13531681e-01
 4.15291273e-01 6.71767330e-01 7.70523337e-01 3.72879320e-01
 6.85258651e-01 5.97205926e-01 8.31775710e-01 4.23683551e-01
 7.22733349e-01 8.35692044e-02 2.30596330e-01 2.41426880e-01
 5.43136613e-01 4.44868536e-01 5.28540519e-01 2.77622762e-01
 1.50708046e-01 7.28094073e-01 9.61917347e-03 7.96781275e-01
 5.76642519e-01 1.61455771e-01 8.99514828e-01 2.07745051e-01
 2.72074972e-02 2.17556087e-01 7.50652298e-01 5.50742758e-01
 7.44551334e-02 6.29450756e-01 8.24222150e-01 3.58440855e-01
 8.75754653e-01 5.54061560e-01 3.12656268e-01 3.57710

# Ant Colony Optimization

In [81]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

input_size = 20
hidden_size = 10
output_size = 1

weights_1 = []
for name, param in net.named_parameters():
    if 'weight' in name:
        weights_1.append(param.data.tolist())

max_weight = 15

values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]
# weights = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]

weights = []
for _ in range(11):
  weights.append(random.randint(1, max_weight))

num_ants = 10
evaporation_rate = 0.1
alpha = 1.0
beta = 2.0
pheromone_init = 0.1


num_items = len(values)
pheromone = np.ones((num_items, max_weight+1)) * pheromone_init


def fitness(solution):
    sum = 0
    lst = [weights[i]*solution[i] for i in range(num_items)]
    for i in lst : 
      sum += i
    total_weight = sum
    if total_weight > max_weight:
        return 0
    
    sum = 0
    lst = [values[i]*solution[i] for i in range(num_items)]
    for i in lst : 
      sum += i
    total_weight = sum
    return sum


def item_selection(ant, remaining_weight):
    item_scores = []
    for i in range(num_items):
        if ant['solution'][i] == 0 and weights[i] <= remaining_weight:
            pheromone_score = pheromone[i][remaining_weight]
            heuristic_score = values[i]/weights[i]
            item_scores.append((i, pheromone_score**alpha * heuristic_score**beta))
    if not item_scores:
        return None
    lst = [score for _, score in item_scores]
    sum = 0
    for i in lst :
      sum += i
    total_score = sum
    probabilities = [(i, score/total_score) for i, score in item_scores]
    selected_item = None
    rand = random.random()
    for i, prob in probabilities:
        rand -= prob
        if rand <= 0:
            selected_item = i
            break
    return selected_item


def update_pheromone(ants):
    for i in range(num_items):
        for j in range(max_weight+1):
            pheromone[i][j] *= (1 - evaporation_rate)

            sum = 0
            lst = [ant['delta_pheromone'][i][j] for ant in ants]
            for i in lst : 
              sum += i
            total_weight = sum
            
            pheromone[i][j] += sum


best_fitness = 0
for iteration in range(100):
    ants = []
    total_fitness = 0
    for ant_index in range(num_ants):
        ant = {'solution': [0]*num_items, 'delta_pheromone': np.zeros((num_items, max_weight+1))}
        remaining_weight = max_weight
        while True:
            item = item_selection(ant, remaining_weight)
            if item is None:
                break
            ant['solution'][item] = 1
            ant['delta_pheromone'][item][remaining_weight] += fitness(ant['solution'])
            remaining_weight -= weights[item]
        ant_fitness = fitness(ant['solution'])
        total_fitness += ant_fitness
        if ant_fitness > best_fitness:
            best_fitness = ant_fitness
            best_solution = ant['solution']
        ants.append(ant)
    update_pheromone(ants)
    avg_fitness = total_fitness / num_ants
    print("Iteration {}: Avg Fitness = {}".format(iteration+1, avg_fitness))

Iteration 1: Avg Fitness = 37.2
Iteration 2: Avg Fitness = 36.3
Iteration 3: Avg Fitness = 34.9
Iteration 4: Avg Fitness = 37.9
Iteration 5: Avg Fitness = 35.0
Iteration 6: Avg Fitness = 36.6
Iteration 7: Avg Fitness = 37.1
Iteration 8: Avg Fitness = 32.0
Iteration 9: Avg Fitness = 32.5
Iteration 10: Avg Fitness = 36.1
Iteration 11: Avg Fitness = 32.9
Iteration 12: Avg Fitness = 37.7
Iteration 13: Avg Fitness = 35.4
Iteration 14: Avg Fitness = 35.8
Iteration 15: Avg Fitness = 31.1
Iteration 16: Avg Fitness = 36.3
Iteration 17: Avg Fitness = 35.8
Iteration 18: Avg Fitness = 37.1
Iteration 19: Avg Fitness = 36.5
Iteration 20: Avg Fitness = 34.4
Iteration 21: Avg Fitness = 36.6
Iteration 22: Avg Fitness = 33.3
Iteration 23: Avg Fitness = 36.1
Iteration 24: Avg Fitness = 38.1
Iteration 25: Avg Fitness = 35.0
Iteration 26: Avg Fitness = 36.1
Iteration 27: Avg Fitness = 35.6
Iteration 28: Avg Fitness = 34.6
Iteration 29: Avg Fitness = 37.3
Iteration 30: Avg Fitness = 37.5
Iteration 31: Avg F

In [70]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

input_size = 20
hidden_size = 10
output_size = 1

weights_1 = []
for name, param in net.named_parameters():
    if 'weight' in name:
        weights_1.append(param.data.tolist())

weights = []
sum = 0
for i in weights_1 :
  for j in i :
    for k in j :
      sum += k
    weights.append(sum)

weights

[11.358983531594276,
 22.199279807507992,
 34.72416888922453,
 46.6854255720973,
 57.15083331614733,
 67.87197101861238,
 79.21215907484293,
 88.99147949367762,
 98.39448668807745,
 108.43941444903612,
 113.16909411083907]